<a href="https://colab.research.google.com/github/konstantinos-al/ml_project_quebec/blob/main/Montelaki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
shapes = []
for image in imgs:
    shapes.append([image.shape[0], image.shape[1]])

shapes = np.array(shapes)

xmin, ymin = shapes.min(axis=0)
xmax, ymax = shapes.max(axis=0)




images = np.array(imgs)
print(images.shape)


trainX, testX, trainY, testY = train_test_split(images, labels, test_size=0.20, stratify=labels, random_state=42)
print(trainX.shape, testX.shape, trainY.shape, testY.shape)
print(trainX.dtype, testX.dtype, trainY.dtype, testY.dtype)



trainX, testX, trainY, testY = train_test_split(images, labels, test_size=0.20, stratify=labels, random_state=42)
# print(trainX.shape, testX.shape, trainY.shape, testY.shape)

# from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical


num_classes = 2

trainY = to_categorical(trainY, num_classes)
trainY = np.array(trainY, dtype='int32')

testY = to_categorical(testY, num_classes)
testY = np.array(testY, dtype='int32')

print(trainX.shape, testX.shape, trainY.shape, testY.shape)

print(trainY)
INIT_LR = 1e-4
EPOCHS = 20
BS = 8


aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(max_height, max_width, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)


model = Model(inputs=baseModel.input, outputs=headModel)


for layer in baseModel.layers:
    layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

print("[INFO] training head...")
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testY.argmax(axis=1), predIdxs))
